# Лекция №11: Математический анализ в Python: Интегрирование

### Цели лекции:
1.  **Понять разницу** между аналитическим (символьным) и численным интегрированием.
2.  **Научиться находить первообразные** и вычислять точные значения интегралов с помощью библиотеки **SymPy**.
3.  **Изучить принципы численных методов:** разобрать логику метода трапеций и метода Симпсона.
4.  **Освоить инструменты SciPy:** научиться вычислять определенные и кратные (двойные, тройные) интегралы с помощью функций из `scipy.integrate`.

## Часть 1. Аналитический vs. Численный подход

При решении задачи нахождения интеграла $\int_{a}^{b} f(x) \,dx$ можно использовать два принципиально разных подхода.

### 1.1. Аналитическое (символьное) интегрирование

Это "классический" подход, знакомый из курса математического анализа. 

1.  **Найти первообразную** $F(x)$ для функции $f(x)$ (т.е. такую функцию, что $F'(x) = f(x)$).
2.  **Применить формулу Ньютона-Лейбница:** $\int_{a}^{b} f(x) \,dx = F(b) - F(a)$.

**Плюсы:**
-   Дает **точное** математическое решение.
-   Позволяет получить ответ в виде формулы, а не числа.

**Минусы:**
-   Для многих функций первообразную невозможно выразить через элементарные функции (например, для $e^{-x^2}$).
-   Может быть очень трудоемким.

**Инструмент в Python:** библиотека **SymPy**.

### 1.2. Численное интегрирование

Этот подход не пытается найти первообразную. Вместо этого он вычисляет **приближенное** значение интеграла.

**Основная идея:** Вспомним, что определенный интеграл — это площадь под кривой. Численные методы аппроксимируют (приближают) эту площадь с помощью простых геометрических фигур (прямоугольников, трапеций и т.д.).

**Плюсы:**
-   Работает для **любой** непрерывной функции, даже если ее первообразная неизвестна.
-   Быстро и эффективно вычисляется на компьютере.

**Минусы:**
-   Дает **приближенный** результат с некоторой погрешностью.

**Инструмент в Python:** библиотека **SciPy** (в частности, модуль `scipy.integrate`).

## Часть 2. Символьное интегрирование с SymPy

**SymPy** — это библиотека для символьных вычислений. Она работает с математическими выражениями как с объектами, а не просто с числами.

In [ ]:
import sympy as sp

# 1. Объявляем символьную переменную
x = sp.Symbol('x')

# 2. Задаем функцию
f = x**2 + sp.cos(x)
f

### 2.1. Неопределенный интеграл (поиск первообразной)

Для этого используется функция `sp.integrate()`.

In [ ]:
# Находим неопределенный интеграл от f
F = sp.integrate(f, x)
print("Первообразная для f:")
F

### 2.2. Определенный интеграл

Чтобы вычислить определенный интеграл, нужно передать в `sp.integrate()` кортеж с переменной, нижней и верхней границами.

In [ ]:
# Вычисляем интеграл от 0 до pi
definite_integral = sp.integrate(f, (x, 0, sp.pi))
print("Точное значение интеграла:")
definite_integral

In [ ]:
# Если нужно числовое значение, используем .evalf()
print("Приближенное значение:")
definite_integral.evalf()

## Часть 3. Численное интегрирование

Рассмотрим, как работают самые простые численные методы, и как использовать мощные инструменты из библиотеки SciPy.

### 3.1. Метод трапеций

**Идея:** Разбить область под кривой на $N$ маленьких трапеций и сложить их площади. Чем больше трапеций (чем меньше их ширина $h$), тем точнее будет результат.

Площадь одной трапеции: $S_i = \frac{f(x_i) + f(x_{i+1})}{2} \cdot h$.

**Реализация "с нуля":**

In [ ]:
import numpy as np

def f_numpy(x):
    return x**2 + np.cos(x)

def trapezoid_method(f, a, b, n):
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    y = f(x)
    integral = h * (np.sum(y) - (y[0] + y[-1]) / 2)
    return integral

a, b = 0, np.pi
n = 1000 # Количество разбиений

result = trapezoid_method(f_numpy, a, b, n)
print(f"Результат методом трапеций (n={n}): {result}")

### 3.2. Метод Симпсона

**Идея:** Вместо того чтобы аппроксимировать кривую прямыми линиями (как в методе трапеций), метод Симпсона использует параболы для аппроксимации. Это дает значительно более высокую точность при том же количестве разбиений.

### 3.3. `scipy.integrate.quad` — "Швейцарский нож"

`quad` (от *quadrature*) — это основная функция для вычисления определенных интегралов в SciPy. Она использует более сложные и точные адаптивные алгоритмы.

**Синтаксис:** `quad(func, a, b)`
-   `func`: подынтегральная функция.
-   `a`, `b`: нижняя и верхняя границы.

Функция возвращает кортеж: `(результат, оценка_погрешности)`.

In [ ]:
from scipy.integrate import quad

# Используем ту же numpy-функцию f_numpy
result, error = quad(f_numpy, 0, np.pi)

print(f"Результат с помощью scipy.quad: {result}")
print(f"Оценка погрешности: {error}")

`quad` легко справляется с бесконечными пределами интегрирования.

In [ ]:
# Вычислим интеграл Гаусса: ∫(-inf, +inf) e^(-x^2) dx = sqrt(pi)
gaussian_func = lambda x: np.exp(-x**2)

result, _ = quad(gaussian_func, -np.inf, np.inf)

print(f"Численное значение интеграла Гаусса: {result}")
print(f"Точное значение (sqrt(pi)): {np.sqrt(np.pi)}")

## Часть 4. Кратные (многократные) интегралы

SciPy также позволяет вычислять двойные и тройные интегралы с помощью функций `dblquad` и `tplquad`.

### 4.1. Двойной интеграл `dblquad`

**Синтаксис:** `dblquad(func, a, b, gfun, hfun)`

Вычисляет $\int_{a}^{b} \int_{gfun(x)}^{hfun(x)} f(x, y) \,dy \,dx$.

-   `func(y, x)`: Подынтегральная функция. **Внимание на порядок аргументов: сначала `y`, потом `x`!**
-   `a`, `b`: Пределы интегрирования по **внешней** переменной (`x`).
-   `gfun`, `hfun`: Функции, задающие пределы интегрирования по **внутренней** переменной (`y`).

**Пример:** Вычислить $\int_{0}^{1} \int_{0}^{1-x} (x+y) \,dy \,dx$.

In [ ]:
from scipy.integrate import dblquad

# Подынтегральная функция f(y, x)
f = lambda y, x: x + y

# Пределы по x
a, b = 0, 1

# Пределы по y: от 0 до 1-x
g = lambda x: 0
h = lambda x: 1 - x

result, _ = dblquad(f, a, b, g, h)
print(f"Значение двойного интеграла: {result}")
# Аналитическое решение также дает 1/3

### 4.2. Тройной интеграл `tplquad`

Работает по аналогии с `dblquad`, но для трех переменных.

**Синтаксис:** `tplquad(func, a, b, gfun, hfun, qfun, rfun)`

Вычисляет $\int_{a}^{b} \int_{gfun(x)}^{hfun(x)} \int_{qfun(x,y)}^{rfun(x,y)} f(x, y, z) \,dz \,dy \,dx$.

-   `func(z, y, x)`: Подынтегральная функция. **Порядок: `z`, `y`, `x`!**
-   `a`, `b`: Пределы по `x`.
-   `gfun`, `hfun`: Пределы по `y` (могут зависеть от `x`).
-   `qfun`, `rfun`: Пределы по `z` (могут зависеть от `x` и `y`).

## Итог

Экосистема Python предоставляет мощные инструменты для решения задач математического анализа:
1.  **SymPy** — для точных, **символьных** вычислений, когда вам нужна формула или аналитическое решение.
2.  **SciPy** — для быстрых и надежных **численных** вычислений, особенно для функций, которые не интегрируются аналитически, или для кратных интегралов.

Понимание обоих подходов и умение выбирать правильный инструмент для конкретной задачи — важный навык для любого инженера или специалиста по данным.